#**Answer Generation 任務**
資料集引用台達閱讀理解資料集









### pytorch環境設定(https://pytorch.org)
選擇適合的版本並複製指令貼到本專案

In [1]:
!pip install torch torchvision

In [2]:
#確定有安裝成功
import torch
print(torch.__version__)
torch.cuda.is_available()

1.5.0+cu101


True

### **取得 google drive 存取權限**

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
cd /content/drive/My Drive/Colab Notebooks/Bert/AnswerGeneration/

/content/drive/My Drive/Colab Notebooks/Bert/AnswerGeneration


In [5]:
!ls

albert		  predict.ipynb  trained_model	 trained_model3  train.json
jsontoList.ipynb  test.json	 trained_model2  train.ipynb


In [0]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

##下載Albert並放在albert目錄

In [7]:
!git clone https://github.com/harry83528/albert-zh-for-pytorch-transformers.git albert

fatal: destination path 'albert' already exists and is not an empty directory.


In [0]:
def use_model(model_name, config_file_path, model_file_path, vocab_file_path, num_labels):
    # 選擇模型並加載設定
    if(model_name == 'bert'):
        from transformers import BertConfig, BertForSequenceClassification, BertTokenizer
        model_config, model_class, model_tokenizer = (BertConfig, BertForSequenceClassification, BertTokenizer)
        config = model_config.from_pretrained(config_file_path,num_labels = num_labels)
        model = model_class.from_pretrained(model_file_path, from_tf=bool('.ckpt' in 'bert-base-chinese'), config=config)
        tokenizer = model_tokenizer(vocab_file=vocab_file_path)
        return model, tokenizer
    elif(model_name == 'albert'):
        from albert.albert_zh import AlbertConfig, AlbertTokenizer, AlbertForMaskedLM
        model_config, model_class, model_tokenizer = (AlbertConfig, AlbertForMaskedLM, AlbertTokenizer)
        config = model_config.from_pretrained(config_file_path)
        #config = model_config.from_pretrained(config_file_path,num_labels = num_labels) # AlbertForMaskedLM 不需分類,不用設num_labels
        model = model_class.from_pretrained(model_file_path, config=config)
        tokenizer = model_tokenizer.from_pretrained(vocab_file_path)
        return model, tokenizer

### **指定想要用的模型是哪一種**
到以下網址查詢模型與函數資訊
https://huggingface.co/transformers

In [0]:
#通常我會在這設一些通用模型參數
# PRETRAINED_MODEL_Name ="bert-base-chinese"  # 指定 想要用的預訓練模型
trained_Model_Path = "trained_model3" #選擇您儲存的model路徑

## **引入需要⽤到的函式庫**

In [0]:
import os
import pickle
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F # 激勵函數
import time

## **預測**

In [0]:
#predict 時把輸入的⽂字轉成輸入格式
def to_input_id(sentence_input):
    #return tokenizer.build_inputs_with_special_tokens(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentence_input))) # 此任務我們自己組special_token
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentence_input))

In [12]:
#設定模型參數 Start =============================================================
#Step1:初始化加載模型 Start======================================================
model_setting = {
    "model_name":"albert", 
    "config_file_path": trained_Model_Path + "/config.json", 
    "model_file_path": trained_Model_Path + "/pytorch_model.bin", 
    "vocab_file_path":"albert/albert_tiny/vocab.txt",
    "num_labels": -1 # 分幾類(AlbertForMaskedLM 不需分類,不用設num_labels,在這隨便亂設)
}
#初始化加載模型
model, tokenizer = use_model(**model_setting)
#Step1:初始化加載模型 END========================================================

#Step2:指定硬體裝置 Start========================================================
# setting device
#你電腦的 GPU 能否被 PyTorch 調用,如果不行就使用CPU 
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("using device",device)
model.to(device)
#Step2:指定硬體裝置 END==========================================================

model.eval()

#print("輸入您想輸入的context")
context ="倭國武周時期改稱日本，與唐朝來往密切。孝德天皇推行革新，效法唐制，走向中央集權。引入均田制和租庸調制，落實戶籍和記帳制度，參考《唐令》寫成《大寶令》法典，遵照長安城布局規劃平安、平城二京。日本先後派遣了十三次遣唐使，每次使團規模都在百人以上，團中除使臣、水手外，還有留學生、學問僧、醫師、音聲生、玉生、鍛生、鑄生、細工生等。著名的來唐日本人有留學生吉備真備和阿倍仲麻呂與僧人空海和圓仁。空海著有《文鏡秘府論》與日本的第一部漢字字典《篆隸萬象名義》。圓仁尋覓佛法而走遍唐國多個道郡，帶回日本大量佛學經文器具。百濟藝僧味摩之將在唐學到的荊楚儺舞傳至日本，稱吳伎樂。日本的文字平假名和片假名也都是分別從中國的草書和楷書部首演變而來。鑒真和尚應日本僧人之邀，曾經六次東渡回日，最後終於成功。他帶去了佛經，促進了中國文化向日本的流傳以及佛教在日本的興盛。"
#context = input()
#print("輸入您想輸入的question")
#question = input()
question ="《篆隸萬象名義》的作者為哪國人？"
input_id = to_input_id("[CLS]" + context + "[SEP]" + question + "[SEP][MASK]")

k = 20 # 取前k個有可能的答案

answer = ""
masked_index = len(input_id)-1 # 要預測的第一個字的索引

# 補齊長度512
while len(input_id)<512:
  input_id.append(0)

#假設答案長度最多15
count = 0
while(count < 15):
    print('input_id:',len(input_id),input_id)
    input_id_tensor=torch.tensor([input_id], dtype=torch.long)
    input_id_tensor = input_id_tensor.to(device) # 資料從CPU RAM 推到 GPU RAM
    outputs = model(input_id_tensor)
    #print('outputs',outputs)

    predictions = outputs[0]
    probs, indices = torch.topk(torch.softmax(predictions[0, masked_index], -1), k)
    indicesList=indices.tolist()
    print('indicesList:',indicesList)
    predicted_tokens = tokenizer.convert_ids_to_tokens(indicesList)
    print('predicted_tokens',type(predicted_tokens),predicted_tokens)

    if predicted_tokens[0] == '[SEP]': #當生成出[SEP]結束
        break

    answer = answer + predicted_tokens[0]
    # 串接預測到的字並再最後的位置補上<MASK>的id 103
    input_id[masked_index] = indicesList[0]
    masked_index = masked_index + 1
    
    if masked_index < 512:
        input_id[masked_index] = 103
    else:
        break

    count += 1
    print('==='*20)

print("生成的答案:" + answer)

using device cuda
input_id: 512 [101, 963, 1751, 3636, 1453, 3229, 3309, 3121, 4935, 3189, 3315, 8024, 5645, 1538, 3308, 889, 2518, 2166, 1147, 511, 2105, 2548, 1921, 4640, 2972, 6121, 7484, 3173, 8024, 3126, 3791, 1538, 1169, 8024, 6624, 1403, 704, 1925, 7415, 3609, 511, 2471, 1057, 1772, 4506, 1169, 1469, 4909, 2435, 6310, 1169, 8024, 5862, 2179, 2786, 5093, 1469, 6250, 2379, 1169, 2428, 8024, 1347, 5440, 517, 1538, 808, 518, 2183, 2768, 517, 1920, 2188, 808, 518, 3791, 1073, 8024, 6905, 4212, 7269, 2128, 1814, 2357, 2229, 6211, 1205, 2398, 2128, 510, 2398, 1814, 753, 776, 511, 3189, 3315, 1044, 2527, 3836, 6897, 749, 1282, 676, 3613, 6897, 1538, 886, 8024, 3680, 3613, 886, 1757, 6211, 3563, 6963, 1762, 4636, 782, 809, 677, 8024, 1757, 704, 7370, 886, 5628, 510, 3717, 2797, 1912, 8024, 6917, 3300, 4522, 2119, 4495, 510, 2119, 1558, 1014, 510, 7015, 2374, 510, 7509, 5476, 4495, 510, 4373, 4495, 510, 7104, 4495, 510, 7139, 4495, 510, 5169, 2339, 4495, 5023, 511, 5865, 1399, 4638, 889, 

In [13]:
print('predict end')

predict end
